# PPG analysis

In [1]:
import importlib.resources
import pickle
from pathlib import Path

from paradigma.config import PPGConfig, IMUConfig, HeartRateConfig
from paradigma.preprocessing import preprocess_ppg_data
from paradigma.pipelines.heart_rate_pipeline import extract_signal_quality_features, signal_quality_classification, estimate_heart_rate, aggregate_heart_rate
from paradigma.util import load_tsdf_dataframe

In [ ]:
# Cell has the tag 'parameters'
branch = 'heart_rate'
sensor = 'ppg'

ppg_prefix = 'PPG'
imu_prefix = 'IMU'

ppg_classifier_package_filename = 'ppg_quality_classifier.pkl'

path_to_data =  Path('../../tests/data')
path_to_assets = Path('../../src/paradigma/assets')

path_to_classifier = path_to_data / '0.Classification' / sensor
path_to_prepared_data = path_to_data / '1.prepared_data'
path_to_preprocessed_data = path_to_data / '2.preprocessed_data' / sensor
path_to_extracted_features = path_to_data / '3.extracted_features' / branch
path_to_predictions = path_to_data / '4.predictions' / branch
path_to_quantification = path_to_data / '5.quantification' / branch
path_to_aggregation = path_to_data / '6.aggregation' / branch

aggregation_filename = 'heart_rate_aggregates.json'

## Load the data

In [ ]:
df_ppg, _, _ = load_tsdf_dataframe(path_to_prepared_data / ppg_prefix, prefix = ppg_prefix)
df_imu, _, _ = load_tsdf_dataframe(path_to_prepared_data / imu_prefix, prefix = imu_prefix)

## Preprocessing

In [ ]:
ppg_config = PPGConfig()
imu_config = IMUConfig()

df_ppg_proc, df_acc_proc = preprocess_ppg_data(df_ppg, df_imu, ppg_config, imu_config)

## Extract signal quality features

In [ ]:
config_ppg = HeartRateConfig(sensor='ppg')
config_acc = HeartRateConfig(sensor='imu')

df_features = extract_signal_quality_features(config_ppg, df_ppg_proc, config_acc, df_acc_proc)

## Signal quality classification

In [ ]:
full_path_to_classifier_package = path_to_assets / ppg_classifier_package_filename
config = HeartRateConfig()
df_sqa = signal_quality_classification(df_features, config, full_path_to_classifier_package)
df_sqa

## Heart rate estimation

In [ ]:
config = HeartRateConfig()
df_hr = estimate_heart_rate(df_sqa, df_ppg_proc, config)

## Heart rate aggregation

In [ ]:
hr_values = df_hr['heart_rate'].values
df_hr_agg = aggregate_heart_rate(hr_values, aggregates = ['mode', '99p'])